# Tweet processing and sentiment Analysis for BTC prediction task

This notebook imports, processes and extracts the sentiment from tweets. Usage of this dataset can be found in the notebook 'Data_Final.ipynb'.

The dataset was taken from Kaggle and can be found [here](https://www.kaggle.com/datasets/kaushiksuresh147/bitcoin-tweets).

In [ ]:
!pip install emoji
!pip install swifter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 6.7 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=c318173326691ef9f23acde6eae8743fcbd9affddd57f5fca7553d8a46568207
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import kagglehub
import nltk
from nltk import TweetTokenizer
import emoji
import swifter
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import gc

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
#import path from Kaggle
path = kagglehub.dataset_download("kaushiksuresh147/bitcoin-tweets")

100%|██████████| 695M/695M [00:37<00:00, 19.7MB/s]

Extracting files...


In [ ]:
#load data in chunks for efficiency and then concatonate
df0 = pd.read_csv(path + '/Bitcoin_tweets.csv',chunksize=100000,lineterminator='\n', low_memory =False)
df = pd.concat(df0)

Some quick initial EDA

In [ ]:
#Initial EDA
df.describe()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
count,4689243,2352532,4169365,4689354,4689354.0,4689354.0,4689354.0,4689354,4689354,4689354,4671838,4685271,4688536
unique,653022,100599,683981,620891,92485.0,34319.0,163739.0,70,3668409,4570199,1013167,3108,1
top,Live Price Crypto,United States,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-03-10 14:06:46,0.0,0.0,0.0,False,2022-05-31 06:02:30,💸 Earn free #BTC and multiply crypto up to 15%...,['Bitcoin'],Twitter for Android,False
freq,41701,63086,29410,43465,60579.0,115000.0,181174.0,3066892,373,1351,608574,1281003,4688536


In [ ]:
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4689354 entries, 0 to 4689353
Data columns (total 13 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_name         object
 1   user_location     object
 2   user_description  object
 3   user_created      object
 4   user_followers    object
 5   user_friends      object
 6   user_favourites   object
 7   user_verified     object
 8   date              object
 9   text              object
 10  hashtags          object
 11  source            object
 12  is_retweet        object
dtypes: object(13)
memory usage: 465.1+ MB


In [ ]:
df.isnull().sum()

,0
user_name,111
user_location,2336822
user_description,519989
user_created,0
user_followers,0
user_friends,0
user_favourites,0
user_verified,0
date,0
text,0


## Converting the Dates, Grouping and Sorting.

In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

#convert to US/Eastern to allign with alpha_vantage
df['date'] = df['date'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

df['date'] = df['date'].dt.date

In [ ]:
df

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4689349,TAnotepad,Not financial advice,#Bitcoin | Forever Alone Crypto Trader 😭 | Ana...,2022-11-04 10:18:11,674,1789,27466,False,2023-01-06,"Bitcoin squeeze is SUPER TIGHT, which way will...","['BTC', 'bitcoin', 'Crypto', 'cryptocurrency',...",Twitter for Android,False
4689350,Boba-Feh,South of the North Pole,My backpack has jets. Reformed Necromonger,2008-04-02 21:48:47,79,454,125,False,2023-01-06,Closed #BTC short at 16725. Missed my long pla...,['BTC'],Twitter for Android,False
4689351,Ethereum Yoda,NaN,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24 04:50:18,532,1,0,False,2023-01-06,#Ethereum price update: \n\n#ETH $1263.59 USD\...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False
4689352,Bitcoin Price Ticker,NaN,Tweets the current price of #bitcoin every 5 m...,2022-10-20 07:10:38,83,7,9,False,2023-01-06,1₿ = $16814.7 -0.07%🔻\n\nDetails:\nChange: 🔻-1...,"['bitcoin', 'btc']",BitcoinLivePriceTicker,False


In [ ]:
#sort dataframe by date
df = df.sort_values(by='date', ascending=True)
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
19595,Alexis Butler,"Colorado, USA",Helping agency owners hit 7-figures in 2021. 💰...,2012-05-29 16:09:25,178.0,14,210,False,2021-02-05,"Last Week I Broke a Record &amp; Made $15,000 ...",NaN,Twitter for iPhone,False
20025,Hunar Campus,NaN,Skills Redefined,2021-01-05 10:18:50,0.0,1,0,False,2021-02-05,Last business model to be discussed is that of...,"['wazirxwarriors', 'indiawantscrypto']",Twitter Web App,False
20026,Ether🩸Reaper,NaN,Founder: Reaper’s🏔Valhalla / #NFT artist / cry...,2015-06-08 09:18:38,1721.0,621,35273,False,2021-02-05,"I am pleased to present, The Digging Badger. \...","['NFT', 'ETH', 'BTC', 'Bitcoin', 'Ethereum']",Twitter for iPhone,False
20027,#MakeADifference,Online,Are you ready to make a difference?\n\nGet $10...,2015-05-02 16:54:25,78779.0,46656,120855,False,2021-02-05,https://t.co/h0XWfZiGQM Market Wrap: #Bitcoin ...,['Bitcoin'],Twibble.io,False
20028,𝘾𝙧𝙮𝙥𝙩𝙤𝙃𝙪𝙣𝙩𝙚𝙧,NaN,Technical Analyst ° Crypto Trader ° Swing & Sc...,2018-01-04 16:16:20,16265.0,79,8790,False,2021-02-05,#unfi $unfi Easy money ?\n\n#btc #bitcoin http...,"['unfi', 'btc', 'bitcoin']",Twitter Web App,False


In [ ]:
df_day = df.groupby('date').count() #Counts the number of entries in each group, returns the number of non-null entries for each group.
df_day

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,text,hashtags,source,is_retweet
date,,,,,,,,,,,,
2021-02-05,2237,1298,2048,2237,2237,2237,2237,2237,2237,1655,2218,2237
2021-02-06,3249,1946,3029,3249,3249,3249,3249,3249,3249,2474,3214,3249
2021-02-07,2915,1777,2670,2915,2915,2915,2915,2915,2915,2230,2878,2915
2021-02-08,6242,3761,5842,6242,6242,6242,6242,6242,6242,5136,6187,6242
2021-02-09,3954,2317,3670,3955,3955,3955,3955,3955,3955,3080,3915,3955
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,17069,8169,15480,17069,17069,17069,17069,17069,17069,17069,17069,17069
2023-01-06,7013,2986,6559,7014,7014,7014,7014,7014,7014,7014,7014,7014
2023-01-07,20114,9355,18637,20114,20114,20114,20114,20114,20114,20114,20114,20114


We can see here that we have 224 dates. The range 2021/02/05	-> 2023/01/09 should have 704 days, meaning we are missing 480. This is only a third of the dates represented in the dataset. Therefore we will take a smaller subset as 'imputing' a smaller subset, hopefully means filling smaller gaps.

In [ ]:
df['date'].dtype

dtype('O')

In [ ]:
#creating subset of data
start_range = pd.to_datetime('2022-01-01')
end_range = pd.to_datetime('2022-06-30')


df['date'] = pd.to_datetime(df['date'])
df_short_term = df[(df['date'] >= start_range) & (df['date'] <= end_range)] #.copy()
df_short_term.head(10)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
2111285,satsleft.info,The Bitcoin Timechain,Reporting on the remaining #Bitcoin supply to ...,2021-10-14 21:31:42,230.0,362.0,37.0,False,2022-01-11,👋 A new block was found on the #Bitcoin networ...,"['Bitcoin', 'bitcoin', 'sats']",satsleft,False
2111282,Sandbaad2,NaN,NaN,2021-11-29 14:37:03,212.0,589.0,10717.0,False,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...,"['BlueSparrow', 'BestDrawToken', 'web3', 'Bitc...",Twitter for Android,False
2111283,₿TC ₿lock₿ot,Ubuntu Bx Undisclosed Location,Keeping you up to date on the latest BTC Block...,2019-11-21 22:08:19,359.0,72.0,34.0,False,2022-01-11,"$BTC Latest Block Info: Block 718220 holds 2,5...",['Bitcoin'],BTC_BlockBot,False
2111284,tictoknextblock,NaN,Real-time bitcoin insights,2021-07-01 03:29:10,174.0,11.0,1.0,False,2022-01-11,"🟩 Block 718220 ⛏ ⛏\n\n📅 11th Jan 2022, 22:12:3...","['bitcoin', 'tictoknextblock']",tictoknextblock,False
2111281,Sandbaad2,NaN,NaN,2021-11-29 14:37:03,212.0,589.0,10717.0,False,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...,"['BlueSparrow', 'BestDrawToken', 'web3', 'Bitc...",Twitter for Android,False
2111286,TokenPrice.cc,Blockchain,https://t.co/qYUjk6bzEq provides fundamental a...,2021-08-26 21:48:29,11923.0,35.0,1202.0,False,2022-01-11,#Bitcoin = BOSS\n\n#tokenpricecc #Crypto #cryp...,"['Bitcoin', 'tokenpricecc', 'Crypto', 'cryptoc...",Twitter Web App,False
2111294,Sandbaad2,NaN,NaN,2021-11-29 14:37:03,212.0,589.0,10717.0,False,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...,"['BlueSparrow', 'BestDrawToken', 'web3', 'Bitc...",Twitter for Android,False
2111288,Maxwell,Massachusetts,Be the change you want to see in the world.\nM...,2021-05-13 01:45:34,1.0,5.0,13.0,False,2022-01-11,@binance My investments for 2022:\n$3K in #bit...,['bitcoin'],Twitter Web App,False
2111289,Sandbaad2,NaN,NaN,2021-11-29 14:37:03,212.0,589.0,10717.0,False,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...,"['BlueSparrow', 'BestDrawToken', 'web3', 'Bitc...",Twitter for Android,False
2111290,Igloobamboo,Hexico,Richard Heart maximalist who missed out on HEX...,2021-08-16 18:02:56,2397.0,2497.0,31741.0,False,2022-01-11,You're going to want to sub this new YouTube c...,"['Bitcoin', 'Ethereum', 'MATIC', 'Chainlink', ...",Twitter for Android,False


In [ ]:
df_short_term.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1819005 entries, 2111285 to 3786421
Data columns (total 13 columns):
 #   Column            Dtype         
---  ------            -----         
 0   user_name         object        
 1   user_location     object        
 2   user_description  object        
 3   user_created      object        
 4   user_followers    object        
 5   user_friends      object        
 6   user_favourites   object        
 7   user_verified     object        
 8   date              datetime64[ns]
 9   text              object        
 10  hashtags          object        
 11  source            object        
 12  is_retweet        object        
dtypes: datetime64[ns](1), object(12)
memory usage: 194.3+ MB


In [ ]:
df_short_term.tail()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
3786418,Chris Taylor,Cape Town,Bitcoin ☀️ \nTravel 🌍\nNadia Taylor ❤,2011-02-20 13:29:26,2995.0,758.0,16003.0,False,2022-06-29,Saudi Saylor is buying #bitcoin 🇸🇦 https://t.c...,['bitcoin'],Twitter for Android,False
3786419,Lucky Station💳,NaN,Consultant Independent en suivi-evaluation des...,2022-04-24 11:19:21,31.0,133.0,0.0,False,2022-06-29,"If #Bitcoin doesn't reach $50,000 by April 20...",['Bitcoin'],Twitter Web App,False
3786420,Ross,EU,"🚀\n 🐱cat moms, \n🍷food, 📸📸photography,\n 🇪🇺tra...",2013-12-27 11:17:48,1680.0,1214.0,4233.0,False,2022-06-29,"After June’s volatility and challenges, #Bitco...",['Bitcoin'],Twitter Web App,False
3786413,mohak,NaN,https://t.co/cmJyjarhsr,2014-10-08 17:44:47,106.0,1511.0,2078.0,False,2022-06-29,Keep Up With Crypto in Just Minutes -\nCut thr...,"['cryptonews', 'nft', 'ether', 'altcoin', 'web...",WordPress.com,False
3786421,Gerald Bull,"Lost,in a world full of Grace.","Eng/Design/Fabrication/& Aero-Rube, 70k hrs of...",2021-05-11 13:46:15,40.0,87.0,1179.0,False,2022-06-29,@TXMCtrades I like #Bitcoin &amp; Im glad U s...,['Bitcoin'],Twitter Web App,False


In [ ]:
df_day_2 = df_short_term.groupby('date').count()


In [ ]:
print(len(df_day_2))

77


As you can see we have 76 unique days, out of 170 meaning we have tweets for 45% of the time period which is an improvement from earlier.
The first and second halves of 2022 were tested.


## Investigating Useful Columns

In [ ]:
print(len(df_short_term))

1819005


In [ ]:
print(df_short_term['is_retweet'].dtype)

object


In [ ]:
df_short_term.isna().sum()

,0
user_name,12
user_location,932243
user_description,217010
user_created,0
user_followers,0
user_friends,0
user_favourites,0
user_verified,0
date,0
text,0


In [ ]:
print(df_short_term['is_retweet'].unique())

[False nan]


In [ ]:
print(df_short_term['user_verified'].unique())

[False True 'False' 'True']


In [ ]:
df_short_term.head(20)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
2111285,satsleft.info,The Bitcoin Timechain,Reporting on the remaining #Bitcoin supply to ...,2021-10-14 21:31:42,230.0,362.0,37.0,False,2022-01-11,👋 A new block was found on the #Bitcoin networ...,"['Bitcoin', 'bitcoin', 'sats']",satsleft,False
2111282,Sandbaad2,NaN,NaN,2021-11-29 14:37:03,212.0,589.0,10717.0,False,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...,"['BlueSparrow', 'BestDrawToken', 'web3', 'Bitc...",Twitter for Android,False
2111283,₿TC ₿lock₿ot,Ubuntu Bx Undisclosed Location,Keeping you up to date on the latest BTC Block...,2019-11-21 22:08:19,359.0,72.0,34.0,False,2022-01-11,"$BTC Latest Block Info: Block 718220 holds 2,5...",['Bitcoin'],BTC_BlockBot,False
2111284,tictoknextblock,NaN,Real-time bitcoin insights,2021-07-01 03:29:10,174.0,11.0,1.0,False,2022-01-11,"🟩 Block 718220 ⛏ ⛏\n\n📅 11th Jan 2022, 22:12:3...","['bitcoin', 'tictoknextblock']",tictoknextblock,False
2111281,Sandbaad2,NaN,NaN,2021-11-29 14:37:03,212.0,589.0,10717.0,False,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...,"['BlueSparrow', 'BestDrawToken', 'web3', 'Bitc...",Twitter for Android,False
2111286,TokenPrice.cc,Blockchain,https://t.co/qYUjk6bzEq provides fundamental a...,2021-08-26 21:48:29,11923.0,35.0,1202.0,False,2022-01-11,#Bitcoin = BOSS\n\n#tokenpricecc #Crypto #cryp...,"['Bitcoin', 'tokenpricecc', 'Crypto', 'cryptoc...",Twitter Web App,False
2111294,Sandbaad2,NaN,NaN,2021-11-29 14:37:03,212.0,589.0,10717.0,False,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...,"['BlueSparrow', 'BestDrawToken', 'web3', 'Bitc...",Twitter for Android,False
2111288,Maxwell,Massachusetts,Be the change you want to see in the world.\nM...,2021-05-13 01:45:34,1.0,5.0,13.0,False,2022-01-11,@binance My investments for 2022:\n$3K in #bit...,['bitcoin'],Twitter Web App,False
2111289,Sandbaad2,NaN,NaN,2021-11-29 14:37:03,212.0,589.0,10717.0,False,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...,"['BlueSparrow', 'BestDrawToken', 'web3', 'Bitc...",Twitter for Android,False
2111290,Igloobamboo,Hexico,Richard Heart maximalist who missed out on HEX...,2021-08-16 18:02:56,2397.0,2497.0,31741.0,False,2022-01-11,You're going to want to sub this new YouTube c...,"['Bitcoin', 'Ethereum', 'MATIC', 'Chainlink', ...",Twitter for Android,False


## Columns to drop
We can see from above 'is_retweet' is either False or NaN, it provides no useful information.
Other columns that should be removed either for anonymity or lack of information are:

*  user_name - ethical
*  user_location - either NaN, not reliable
*  user_description - not helpful here
* user_created
* user_friends - enough information in 'user_followers'
* user_verified
* source
* is_retweet - all False or NaN

In [ ]:
df_short_term = df_short_term.drop(columns=['user_name', 'user_location', 'user_description', 'user_created', 'user_friends', 'user_verified', 'source', 'is_retweet'])

In [ ]:
df_short_term.head()

,user_followers,user_favourites,date,text,hashtags
2111285,230.0,37.0,2022-01-11,👋 A new block was found on the #Bitcoin networ...,"['Bitcoin', 'bitcoin', 'sats']"
2111282,212.0,10717.0,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...,"['BlueSparrow', 'BestDrawToken', 'web3', 'Bitc..."
2111283,359.0,34.0,2022-01-11,"$BTC Latest Block Info: Block 718220 holds 2,5...",['Bitcoin']
2111284,174.0,1.0,2022-01-11,"🟩 Block 718220 ⛏ ⛏\n\n📅 11th Jan 2022, 22:12:3...","['bitcoin', 'tictoknextblock']"
2111281,212.0,10717.0,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...,"['BlueSparrow', 'BestDrawToken', 'web3', 'Bitc..."


In [ ]:
df_short_term.isnull().sum()

,0
user_followers,0
user_favourites,0
date,0
text,0
hashtags,54


In [ ]:
df_short_term['hashtags'].unique()

array(["['Bitcoin', 'bitcoin', 'sats']",
       "['BlueSparrow', 'BestDrawToken', 'web3', 'Bitcoin']",
       "['Bitcoin']", ...,
       "['socialmediamarketing', 'socialmedia', 'affiliatemarketing', 'pinterest', 'workfromhome', 'seo', 'facebook', 'onlinemarketing', 'bitcoin', 'affiliatemarketer']",
       "['bitcoin', 'Crypto', 'wildwest', 'gosayhello', 'fire', 'burn', 'grayscale', '3AC', 'whales', 'bears', 'crooks']",
       "['Bitcoin', 'cryptocurrecy', 'cryptotrade']"], dtype=object)

In [ ]:
#for efficient computing just keep tweet text and date for processing
df_text = df_short_term.drop(columns=['user_followers' ,	'user_favourites' ,	'hashtags'])

In [ ]:
df_text

,date,text
2111285,2022-01-11,👋 A new block was found on the #Bitcoin networ...
2111282,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...
2111283,2022-01-11,"$BTC Latest Block Info: Block 718220 holds 2,5..."
2111284,2022-01-11,"🟩 Block 718220 ⛏ ⛏\n\n📅 11th Jan 2022, 22:12:3..."
2111281,2022-01-11,@ianheinischmma @ShannonBrayNC Huuuuge News!!!...
...,...,...
3786418,2022-06-29,Saudi Saylor is buying #bitcoin 🇸🇦 https://t.c...
3786419,2022-06-29,"If #Bitcoin doesn't reach $50,000 by April 20..."
3786420,2022-06-29,"After June’s volatility and challenges, #Bitco..."
3786413,2022-06-29,Keep Up With Crypto in Just Minutes -\nCut thr...


## Tweet Preprocessing for Sentiment Analysis
*  Use Regex to replace any long URLs and any @Users (anonymity)
*  Use Tweet Tokenizer to tokenize the text
*  Use the emoji package to keep sentiment of emojis
*  Vader used to extract sentiment score

Resources on Applying sentiment analysis and Vader can be found here:
* [Applying Sentiment Analysis](https://www.learndatasci.com/tutorials/sentiment-analysis-reddit-headlines-pythons-nltk/?utm_source=chatgpt.com)
* [VADER](https://hex.tech/templates/sentiment-analysis/vader-sentiment-analysis/#:~:text=VADER%20is%20a%20rule%2Dbased,with%20their%20corresponding%20sentiment%20ratings.)
* [Tweet Tokenizer](https://www.geeksforgeeks.org/python-nltk-nltk-tweettokenizer/)


In [ ]:
#Replacing links and tags
gc.disable() #stop garbage collection
df_text['text'] = df_text['text'].str.replace(r'http\S+', 'URL', regex=True)
df_text['text'] = df_text['text'].str.replace(r'@\S+', '@USER', regex=True)

In [ ]:
df_text.head()

,date,text
2111285,2022-01-11,👋 A new block was found on the #Bitcoin networ...
2111282,2022-01-11,@USER @USER Huuuuge News!!!\nNew Utility Incom...
2111283,2022-01-11,"$BTC Latest Block Info: Block 718220 holds 2,5..."
2111284,2022-01-11,"🟩 Block 718220 ⛏ ⛏\n\n📅 11th Jan 2022, 22:12:3..."
2111281,2022-01-11,@USER @USER Huuuuge News!!!\nNew Utility Incom...


In [ ]:
#preprocessing

#initialize tokenizer
tokenizer = TweetTokenizer()
model = SentimentIntensityAnalyzer()

#preprocessing function
def text_preprocessing(text):
  if pd.isna(text) or text.strip() == '':
    return text

  text = emoji.demojize(text)

  tokens = tokenizer.tokenize(text)

  return tokens


def score_extraction(text):
  score = model.polarity_scores(text)
  compound = score['compound']

  if compound > 0.05:
    sentiment = 'Positive'
  elif compound < -0.05:
    sentiment = 'Negative'
  else:
    sentiment = 'Neutral'

  return sentiment, compound

Applying the Sentiment Analysis

In [ ]:
#apply to chunks and cocatenate for effciency and  use swifter

chunk_size = 50000
chunks = [df_text[i:i+chunk_size].copy() for i in range(0, len(df_text), chunk_size)]

processed_chunks = []

for chunk in chunks:
  chunk['processed_text'] = chunk['text'].swifter.apply(text_preprocessing)
  chunk[['sentiment', 'compound_score']] = chunk['text'].swifter.apply(lambda x: pd.Series(score_extraction(x))).rename({0: 'sentiment', 1: 'compound_score'}, axis=1)
  processed_chunks.append(chunk)

df_processed = pd.concat(processed_chunks, ignore_index=True)

Pandas Apply:   0%|          | 0/50000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50000 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/50000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/19005 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/19005 [00:00<?, ?it/s]

In [ ]:
df_processed.head()

,date,text,processed_text,sentiment,compound_score
0,2022-01-11,👋 A new block was found on the #Bitcoin networ...,"[:, waving_hand, :, A, new, block, was, found,...",Negative,-0.7003
1,2022-01-11,@USER @USER Huuuuge News!!!\nNew Utility Incom...,"[@USER, @USER, Huuuuge, News, !, !, !, New, Ut...",Negative,-0.3530
2,2022-01-11,"$BTC Latest Block Info: Block 718220 holds 2,5...","[$, BTC, Latest, Block, Info, :, Block, 718220...",Negative,-0.7845
3,2022-01-11,"🟩 Block 718220 ⛏ ⛏\n\n📅 11th Jan 2022, 22:12:3...","[:, green_square, :, Block, 718220, :p, ick, :...",Positive,0.2500
4,2022-01-11,@USER @USER Huuuuge News!!!\nNew Utility Incom...,"[@USER, @USER, Huuuuge, News, !, !, !, New, Ut...",Negative,-0.3530


# Creating a dataset just with the day and sentiment score

In [ ]:
#get daily average sentiment score
tweet_daily_sentiment = df_processed.groupby('date').agg(avg_sentiment_score = ('compound_score', 'mean')).reset_index()
print(tweet_daily_sentiment)

         date  avg_sentiment_score
0  2022-01-11             0.210110
1  2022-01-12             0.260047
2  2022-01-13             0.222628
3  2022-01-14             0.253282
4  2022-01-19             0.208571
..        ...                  ...
72 2022-06-24             0.154576
73 2022-06-25             0.179629
74 2022-06-26             0.195231
75 2022-06-28             0.151321
76 2022-06-29             0.158009

[77 rows x 2 columns]


# Saving the dataset for future use

In [ ]:
#saving files to CSV in Google Drive
from google.colab import drive
drive.mount('/content/drive')


#file_path = '/content/drive/MyDrive/ML-project/processed_tweets_final.csv'
file_path_1 = '/content/drive/MyDrive/ML-project/tweet_daily_sentiment_final.csv'

# Save DF as CSV
#df_processed.to_csv(file_path, index=False)
tweet_daily_sentiment.to_csv(file_path_1, index=False)

#print(f"File saved to: {file_path}")
print(f"File saved to: {file_path_1}")

Mounted at /content/drive
File saved to: /content/drive/MyDrive/processed_tweets_final.csv
File saved to: /content/drive/MyDrive/tweet_daily_sentiment_final.csv


We will now use this dataset in 'Data_Final.ipynb'. Note this notebook could be pasted at the beginning of the next notebook. However, due to RAM issues, this is not reccomended. This notebook can be run with CPU/T4 GPU (Set to T4 here).